In [63]:
import sys
import pickle as pk

import nest_asyncio
nest_asyncio.apply()
from ib_insync import Stock, IB, util
import pandas as pd

sys.path[0] = '..'
import config
from QuantumCapital import constants
from QuantumCapital.DBManager import DBManager

In [52]:
dbm = DBManager(config.DB_USERNAME, config.DB_PASS, 'ODS')

In [15]:
ib = IB()
ib.connect('192.168.31.7', 4001, clientId=9, readonly=True)

<IB connected to 192.168.31.7:4001 clientId=9>

In [3]:
with open('../Models_&_Files/Quarter_Reports_Models/rus_combined.dictionary', 'rb') as file:
    rus_tickers = pk.load(file)

In [4]:
rus_tickers = list(rus_tickers.keys())[:10]

In [24]:
ticker = rus_tickers[0]

cols = {'dt': 'timestamp with time zone',
        'open': 'real',
        'high': 'real',
        'low': 'real',
        'close': 'real',
        'volume': 'real'}
dbm.create_table(ticker + constants.MINUTE_BAR, cols)

contract = Stock(ticker, 'SMART/ISLAND', 'USD')
bars = ib.reqHistoricalData(
    contract,
    endDateTime='',
    durationStr='167 D',
    barSizeSetting='1 min',
    whatToShow='TRADES',
    useRTH=True,
    formatDate=2,
    timeout=60)
df = util.df(bars)

In [30]:
df = df[['date', 'open', 'high', 'low', 'close', 'volume']]
df.columns = ['dt', 'open', 'high', 'low', 'close', 'volume']
df

,dt,open,high,low,close,volume
0,2020-07-01 13:30:00+00:00,11.18,11.28,11.15,11.24,801
1,2020-07-01 13:31:00+00:00,11.25,11.27,11.21,11.22,87
2,2020-07-01 13:32:00+00:00,11.22,11.25,11.21,11.22,84
3,2020-07-01 13:33:00+00:00,11.23,11.25,11.21,11.24,54
4,2020-07-01 13:34:00+00:00,11.24,11.31,11.23,11.27,217
...,...,...,...,...,...,...
64765,2021-03-01 20:55:00+00:00,26.04,26.05,26.00,26.01,265
64766,2021-03-01 20:56:00+00:00,26.01,26.03,26.01,26.02,215
64767,2021-03-01 20:57:00+00:00,26.02,26.04,26.00,26.01,262
64768,2021-03-01 20:58:00+00:00,26.00,26.01,25.98,26.00,271


In [54]:
dbm.insert_df(df, ticker + constants.MINUTE_BAR_NAMING)

In [59]:
tt = dbm.select_df(f'select * from {ticker + constants.MINUTE_BAR}')

In [64]:
df['dt'] = df['dt'] + pd.Timedelta(minutes=1)

In [65]:
df

,dt,open,high,low,close,volume
0,2020-07-01 13:31:00+00:00,11.18,11.28,11.15,11.24,801
1,2020-07-01 13:32:00+00:00,11.25,11.27,11.21,11.22,87
2,2020-07-01 13:33:00+00:00,11.22,11.25,11.21,11.22,84
3,2020-07-01 13:34:00+00:00,11.23,11.25,11.21,11.24,54
4,2020-07-01 13:35:00+00:00,11.24,11.31,11.23,11.27,217
...,...,...,...,...,...,...
64765,2021-03-01 20:56:00+00:00,26.04,26.05,26.00,26.01,265
64766,2021-03-01 20:57:00+00:00,26.01,26.03,26.01,26.02,215
64767,2021-03-01 20:58:00+00:00,26.02,26.04,26.00,26.01,262
64768,2021-03-01 20:59:00+00:00,26.00,26.01,25.98,26.00,271


Peer closed connection
Peer closed connection
Peer closed connection


In [8]:
# open, high, low, close, volume, dt

for ticker in rus_tickers:
    if dbm.check_if_table_exists(ticker + constants.MINUTE_BAR):
        # get last bar date
        # fetch from that date
        # insert
        pass
    else:
        cols = {'dt': 'timestamp with time zone',
                'open': 'real',
                'high': 'real',
                'low': 'real',
                'close': 'real',
                'volume': 'real'}
        db_manager.create_table(ticker + constants.MINUTE_BAR, cols)
        
        contract = Stock(ticker, 'SMART/ISLAND', 'USD')
        bars = ib.reqHistoricalData(
            contract,
            endDateTime='',
            durationStr='167 D',
            barSizeSetting='1 min',
            whatToShow='TRADES',
            useRTH=True,
            formatDate=2,
            timeout=60)
        idf = util.df(bars).set_index('date')
        
        
        # fetch from that date or maximum old possible
        # inser
        pass

<class 'str'> aa
<class 'str'> ino
<class 'str'> axp
<class 'str'> vz
<class 'str'> spwh
<class 'str'> ba
<class 'str'> sabr
<class 'str'> pep
<class 'str'> cat
<class 'str'> jpm


In [4]:
import pandas as pd

In [15]:
market_members = 'rus members.csv'

stocks = pd.read_csv(market_members)
display(stocks)
tickers = []

for c in stocks:
    tickers.append(stocks[c])
tickers = list(pd.concat(tickers).dropna().unique())

def start_date(year):
    return '01.01.'+str(year)

def end_date(year):
    if year == 2020:
        return 'TODAY()'
    else:
        return '31.12.'+str(year+1)
    
ticker_years = {}
for t in tickers:
    ticker_years[t] = []
    for y in stocks:
        if t in stocks[y].values:
            ticker_years[t].append(y)
    ticker_years[t] = [min(ticker_years[t]), max(ticker_years[t])]

,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002
0,AA UN Equity,AA UN Equity,WMK UN Equity,COKE UW Equity,COKE UW Equity,COKE UW Equity,CVI UN Equity,CVI UN Equity,EIX UN Equity,EIX UN Equity,EIX UN Equity,EIX UN Equity,EIX UN Equity,EIX UN Equity,EIX UN Equity,EIX UN Equity,EIX UN Equity,STU UN Equity,STU UN Equity
1,INO UW Equity,VZ UN Equity,COKE UW Equity,LORL UW Equity,LORL UW Equity,CVI UN Equity,EIX UN Equity,EIX UN Equity,AEE UN Equity,FE UN Equity,WMK UN Equity,MSFG UW Equity,OHAI UW Equity,FE UN Equity,VALU UQ Equity,VALU UQ Equity,STU UN Equity,VELCF US Equity,VELCF US Equity
2,AXP UN Equity,SPWH UW Equity,CCNE UW Equity,TCI UN Equity,OFLX UQ Equity,INO UW Equity,AEE UN Equity,AEE UN Equity,PNM UN Equity,BFIN UW Equity,FFCH UW Equity,OHAI UW Equity,GAIN UW Equity,VALU UQ Equity,SKY UN Equity,SKY UN Equity,VELCF US Equity,WPSL UN Equity,WPSL UN Equity
3,VZ UN Equity,BA UN Equity,WPG UN Equity,SEB UA Equity,ISRL UR Equity,VZ UN Equity,WMK UN Equity,PNM UN Equity,FE UN Equity,AA UN Equity,MSFG UW Equity,OFLX UQ Equity,FE UN Equity,SKY UN Equity,NST UN Equity,STU UN Equity,CHG UN Equity,1831877D UN Equity,1831877D UN Equity
4,SPWH UW Equity,SABR UW Equity,UNB UQ Equity,VHI UN Equity,BCRHF UN Equity,SPWH UW Equity,DJCO UR Equity,AXP UN Equity,SSSS UR Equity,AXP UN Equity,MGEE UW Equity,GAIN UW Equity,1481878D UN Equity,NST UN Equity,STU UN Equity,VELCF US Equity,WPSL UN Equity,CB UN Equity,CB UN Equity
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3008,ALLY UN Equity,ALLY UN Equity,NaN,NaN,NaN,GTI UN Equity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DTSI UQ Equity,NaN,NaN,NaN
3009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WITS UQ Equity,NaN,NaN,NaN
3010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FORM UQ Equity,NaN,NaN,NaN
3011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GFIG UQ Equity,NaN,NaN,NaN


In [19]:
print(len(ticker_years))

9102
